#Tribute to Andrea Infuso

March 2006Eurosurveillance 11(3):1-2;; DOI: 10.2807/esm.11.03.00602-en

"This special issue of Eurosurveillance is dedicated to the memory of Andrea Infuso, a dear and respected colleague and friend, who died suddenly on 20 September 2005 at the age of 44. Andrea was actively involved in the preparation of this special issue on vaccination and tuberculosis. As EuroTB coordinator since 2000, his knowledge of and contacts with all European experts involved in tuberculosis surveillance in Europe were very valuable in conceiving this thematic issue. The Euroroundup published in this issue, European survey of BCG vaccination policies and surveillance in children, 2005, written by Andrea as first author, is a posthumous publication."https://www.researchgate.net/publication/320203408_Tribute_to_Andrea_Infuso

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Codes by Paul Mooney https://www.kaggle.com/paultimothymooney/what-is-inside-of-the-mueller-report/notebook

In [ ]:
# Import Python Packages
# PyTesseract and Tika-Python for OCR
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import PIL
import os
from os import walk
from shutil import copytree, ignore_patterns
from collections import Counter
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from wand.image import Image as Img
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
#mueller_report = pd.read_csv('../input/data-science-cheat-sheets/Interview Questions/AI Questions.pdf') # one row per line

In [ ]:
# Define helper function for plotting word clouds
def wordCloudFunction(df,column,numWords):
    # adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
    topic_words = [ z.lower() for y in
                       [ x.split() for x in df[column] if isinstance(x, str)]
                       for z in y]
    word_count_dict = dict(Counter(topic_words))
    popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
    popular_words_nonstop = [w for w in popular_words if w not in stopwords.words("english")]
    word_string=str(popular_words_nonstop)
    wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          max_words=numWords,
                          width=1000,height=1000,
                         ).generate(word_string)
    plt.clf()
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
# Define helper function for plotting word bar graphs
def wordBarGraphFunction(df,column,title):
    # adapted from https://www.kaggle.com/benhamner/most-common-forum-topic-words
    topic_words = [ z.lower() for y in
                       [ x.split() for x in df[column] if isinstance(x, str)]
                       for z in y]
    word_count_dict = dict(Counter(topic_words))
    popular_words = sorted(word_count_dict, key = word_count_dict.get, reverse = True)
    popular_words_nonstop = [w for w in popular_words if w not in stopwords.words("english")]
    plt.barh(range(50), [word_count_dict[w] for w in reversed(popular_words_nonstop[0:50])])
    plt.yticks([x + 0.5 for x in range(50)], reversed(popular_words_nonstop[0:50]))
    plt.title(title)
    plt.show()

In [ ]:
# Preview the data folder
inputFolder = '../input/'
for root, directories, filenames in os.walk(inputFolder):
    for filename in filenames: 
        print(os.path.join(root,filename))
        
# Move data to folder with read/write access
outputFolder = '/kaggle/working/pdfs/'
shutil.copytree(inputFolder,outputFolder,ignore=ignore_patterns('*.db'))
for root, directories, filenames in os.walk(outputFolder, topdown=False):
    for file in filenames:
        try:
            shutil.move(os.path.join(root, file), outputFolder)
        except OSError:
            pass
print(os.listdir(outputFolder))

In [ ]:
# Look at page 1
pdf = os.path.join(outputFolder,'Azerbaijan-en-result-104-original.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/Azerbaijan-en-result-104-original.jpg') # intro page to preview later

#PDF to CSV

Convert Page 3 of PDF to CSV (Method 1 of 2: PyTesseract)

In [ ]:
# Parse a PDF file and convert it to CSV using PyTesseract
import pytesseract
pdfimage = Image.open('/kaggle/working/Azerbaijan-en-result-104-original.jpg')
text = pytesseract.image_to_string(pdfimage)  
df = pd.DataFrame([text.split('\n')])

In [ ]:
# Plot WordCloud of page 1
plt.figure(figsize=(10,10))
wordCloudFunction(df.T,0,10000000)
plt.figure(figsize=(10,10))
wordBarGraphFunction(df.T,0,"Most Common Words on Page 1 of Azerbaijan")

In [ ]:
# Parse a PDF file and convert it to CSV using Tika-Python
!pip install tika
import tika
from tika import parser
tika.initVM()
parsed = parser.from_file('/kaggle/working/Azerbaijan-en-result-104-original.jpg') 
text = parsed["content"]
df = pd.DataFrame([text.split('\n')])
df.drop(df.iloc[:, 1:46], inplace=True, axis=1)

In [ ]:
# Convert PDF to JPG and then convert JPG to CSV
# I will do this for Pages 289 to 291 but. I don't know how mny pages has the document. 
# Eventually I should loop through the entire document

# PDF to JPG for p1
pdf = os.path.join(outputFolder,'Azerbaijan-en-result-104-original.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/Azerbaijan-en-result-104-original.jpg')
pdfimage1 = Image.open('/kaggle/working/Azerbaijan-en-result-104-original.jpg')

In [ ]:
# PDF to JPG for p2
pdf = os.path.join(outputFolder,'Azerbaijan-en-result-104-original.pdf[2]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/Azerbaijan-en-result-104-original.jpg')
pdfimage2 = Image.open('/kaggle/working/Azerbaijan-en-result-104-original.jpg')

# PDF to JPG for p3
pdf = os.path.join(outputFolder,'Azerbaijan-en-result-104-original.pdf[3]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/Azerbaijan-en-result-104-original.jpg')
pdfimage3 = Image.open('/kaggle/working/Azerbaijan-en-result-104-original.jpg')

In [ ]:
# Parse a PDF file and convert it to CSV using PyTesseract (p1)
text = pytesseract.image_to_string(pdfimage1)
df = pd.DataFrame([text.split('\n')])
df.drop(df.iloc[:, 27:], inplace=True, axis=1)
df.drop(df.iloc[:, :3], inplace=True, axis=1)
df.columns = range(df.shape[1])

In [ ]:
# Parse a PDF file and convert it to CSV using Tika-Python (p290-291)
tika.initVM()
parsed = parser.from_file('/kaggle/working/Azerbaijan-en-result-104-original.jpg')
parsed2 = parser.from_file('/kaggle/working/Azerbaijan-en-result-104-original.jpg')

text = parsed["content"]
df2 = pd.DataFrame([text.split('\n')])
df2.drop(df2.iloc[:, 1:50], inplace=True, axis=1)
df2.drop(df2.iloc[:, 26:], inplace=True, axis=1)
df2.columns = range(df2.shape[1])

text = parsed2["content"]
df3 = pd.DataFrame([text.split('\n')])
df3.drop(df3.iloc[:, :50], inplace=True, axis=1)
df3.drop(df3.iloc[:, 22:], inplace=True, axis=1)
df3.columns = range(df3.shape[1])

dfcombined = pd.concat([df, df2, df3]) # combine pages 289-291

In [ ]:
#Explore page 3 - Mueller Report. Here I don't know how many pages each pdf have.  
w, h = pdfimage3.size # crop image
pdfimage3.crop((0, 1240, w, h-1300)) # display exerpt of PDF

In [ ]:
# Convert PDF to JPG and then convert JPG to CSV
# I will do this for Pages 289 to 291 but
# Eventually I should loop through the entire document

# PDF to JPG for p1
pdf = os.path.join(outputFolder,'Azerbaijan-en-result-104-original.pdf[1]')
with Img(filename=pdf, resolution=300) as img:
    img.compression_quality = 99
    img.convert("RGBA").save(filename='/kaggle/working/Azerbaijan-en-result-104-original.jpg')
pdfimage1 = Image.open('/kaggle/working/Azerbaijan-en-result-104-original.jpg')

In [ ]:
#Explore page 1 - Mueller Report. Here I don't know how many pages each pdf file have.  
w, h = pdfimage1.size # crop image
pdfimage1.crop((0, 1240, w, h-1300)) # display exerpt of PDF

In [ ]:
# Pages 1, 2, and 3
dfcombined.head() # preview csv of 289-291

In [ ]:
# Clean up the notebook
!apt-get install zip # install zip
!zip -r pdfs.zip /kaggle/working/pdfs/ # zip up a few files
!rm -rf pdfs/* # remove everything else

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#32a832','#32a84e','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks Andrea Infuso, @mpwolke was here' )